# 1) Tide Revisited with internal ML

## a) Data Preprocessing

In [ ]:
import pandas as pd
from klines_managers import klines_ccxt

timeframe_to_trade = "1h"


instruments =  ["ccxt_kucoin__BTC-USDT",
                 "ccxt_currencycom__US500",
                 "ccxt_currencycom__DXY",
                 "ccxt_currencycom__Gold",
                 "ccxt_currencycom__Oil - Crude",
                  "ccxt_currencycom__NVDA",
                   "ccxt_currencycom__UVXY" ]
timeframes = ["1h"]

kline_manager = klines_ccxt.KlinesManagerCCXT()
klines_dict = kline_manager.load_ohlcvs(instruments = instruments,
                                        timeframes = timeframes,
                                        since = "2019-01-01 00:00:00",
                                        limit = 1000, update=False)


# Resample from 1hr --> 2h, 3h, 4h, 6h, 12h, 24h, 
from signal_managers import resampler
from signal_managers import indicators

resample_to_list = ["2h", "3h", "4h", "6h", "12h", "24h"]
first_timeframe = "1h"

print(klines_dict.keys())
print(klines_dict['ccxt_kucoin__BTC-USDT'].keys())
# resample_to_list = ["15m", "20m", "30m", "60m"]
# first_timeframe = "5m"
for instrument,instrument_dict in klines_dict.items():
    # print(f"\n{instrument}\n")
    for resample_to in resample_to_list:
        df = instrument_dict[first_timeframe].copy()
        window = int(int(resample_to[:-1])/int(first_timeframe[:-1]))
        # print(f"Resampling {first_timeframe} to {resample_to} --> window: {window}")
        if "m" in resample_to:
            resample_to_formatted = resample_to[:-1]+"T"
        else:
            resample_to_formatted = resample_to
        df_resampled = resampler.calc_klines_resample(df,window=window, resample_to=resample_to_formatted)
        # print(df_resampled.columns)
        klines_dict[instrument][resample_to] = df_resampled


from signal_managers.indicators import calc_tide_strengths
from signal_managers import tide_genetic

print(klines_dict.keys())
print(klines_dict['ccxt_kucoin__BTC-USDT'].keys())
# generate tides for each instruments
instrument_to_trade = "ccxt_kucoin__BTC-USDT"
timeframe_to_trade = "1h"
for instrument in klines_dict.keys():
    for timeframe in klines_dict[list(klines_dict.keys())[0]]:
        timeframe_multiplier = int(timeframe[:-1])/int(timeframe_to_trade[:-1])
        df = klines_dict[instrument][timeframe]
        df = indicators.calc_tides(df,suffix="")
        df = indicators.calc_z_sig(df,cols=['open','high','low','close'], lookback_bars = 24*timeframe_multiplier, threshold = [])
        df=df.dropna()
        from signal_managers import tide_genetic

        # Initialise parent
        tp_position_list = tide_genetic.init_tp_position_parameter_population()
        df0 = tide_genetic.initialise_parent(df)
        df0 = calc_tide_strengths(df0, penalty = 1, tp_position_dict = tp_position_list[0])
        features = ['tide', 'ebb', 'flow', 'sig'] + list(df0.copy().filter(regex="tide").columns)[:-1]
        features = list(set(features))

        tide_parameters_to_test = {"sensitivity":40,
                                "threshold":10,
                                    "windows":[24,36,72],
                                    "SL_penalty":1,
                                    "TPs": tp_position_list[0],
                                }
        sharpe_df, best_df, best_parameter, best_sharpe = tide_genetic.evaluate_population(df0,parameters = tide_parameters_to_test, generations=10)
        # Perform genetic propagation
        best_df[features] = best_df[features].copy().fillna(method='ffill').dropna()
        klines_dict[instrument][timeframe] = best_df
        
        # Join into one DF
        if instrument == instrument_to_trade and timeframe == timeframe_to_trade:
            df_trade = best_df.copy() 

        else:
            df["sharpe"] = best_sharpe["sharpe"].iloc[0]
            df_trade = pd.merge(df_trade, best_df[features].add_prefix(f"{timeframe}_{instrument.split('__')[-1]}_"), left_index=True, right_index=True, how="left")
    


## a) Tide_Z signal V3

In [1]:
import pandas as pd
from klines_managers import klines_ccxt

timeframe_to_trade = "1h"


instruments =  ["ccxt_kucoin__BTC-USDT",
                 "ccxt_currencycom__US500",
                 "ccxt_currencycom__DXY",
                 "ccxt_currencycom__Gold",
                 "ccxt_currencycom__Oil - Crude",
                  "ccxt_currencycom__NVDA",
                   "ccxt_currencycom__UVXY"]
timeframes = ["1h"]

kline_manager = klines_ccxt.KlinesManagerCCXT()
klines_dict = kline_manager.load_ohlcvs(instruments = instruments,
                                        timeframes = timeframes,
                                        since = "2018-01-01 00:00:00",
                                        limit = 1000, update=False, verbose=False)

df = klines_dict["ccxt_kucoin__BTC-USDT"]["1h"]#.copy() but memwasteful since can just reload the klines again

### i) MFI V3

In [2]:
from signal_managers.indicators import calc_mfi_sig
import numpy as np
def dynamic_param_func(x, i):
    """
    Define a more complex function to calculate the window lengths, thresholds, and sensitivity parameters
    based on characteristics of the input data.

    Args:
        x: x is a np_array of cols + dynamic_param_col
        i: The index of the current data point.

    Returns:
        The list of window lengths to use.
        The list of thresholds to use for each window length.
        The sensitivity parameter.
    """
    std = np.std(x[:,-1])

    # Calculate the minimum window length.
    min_window_length = int(std * 10)

    # Calculate the maximum window length.
    max_window_length = int(std * 100)

    # Calculate the number of thresholds to use.

    # Generate a list of possible window lengths.
    window_lengths = np.linspace(min_window_length, max_window_length, num=10)

    return window_lengths

def dynamic_param_func(x, i):
    """
    Define a more complex function to calculate the window lengths, thresholds, and sensitivity parameters
    based on characteristics of the input data.

    Args:
        x: x is a np_array of cols + dynamic_param_col
        i: The index of the current data point.

    Returns:
        The list of window lengths to use.
        The list of thresholds to use for each window length.
        The sensitivity parameter.
    """
    window_lengths = [13]#,21,34,55,89,144]

    return window_lengths


df0= df.copy()
df1 = calc_mfi_sig(df0,
                   cols_set = [['high','low', 'close', 'volume']],
                   MFI_window_threshold_func = dynamic_param_func,
                   dynamic_param_col= ["volume"],
                   fixed_window = False,)
                                

  0%|          | 0/1 [00:00<?, ?it/s]d:\PycharmProjects\quant_experiments\signal_managers\indicators.py:271: RuntimeWarning: invalid value encountered in scalar divide
  mfi = np.where(np.abs(denominator) > tolerance, numerator / denominator, np.where(np.abs(denominator) < tolerance, np.inf, -np.inf))
  0%|          | 0/1 [01:08<?, ?it/s]


IndexError: index 1 is out of bounds for axis 0 with size 1

### i) Tide V3

In [ ]:
from signal_managers.indicators import calc_tide_sig
import numpy as np
from utils.list_type_converter import convert_to_type,convert_to_array
def dynamic_params_func(x, i):
    # Calculate the volatility of the input data
    try:
        volatility = x[-1] # This should be a EMA vol calculated outside
    except Exception as e:
        volatility = 0
    # print(f"volatility in dynamic_params_func {i}: {volatility}") # This is too smooth to be vol triggers have to change for future use
    # Set the window lengths and threshold values based on the volatility
    if volatility < 0.01:
        windows = [[288]]
        thresholds = [5]
    elif volatility < 0.05:
        windows = [[144]]
        thresholds = [10]
    else:
        windows = [[72, 144]]
        thresholds = [20]
    
    return convert_to_array(windows), convert_to_type(thresholds, int), convert_to_type([0.5], float)

def dynamic_params_func1(x, i):
    """
    Template for dynamic parameters function
    where x is a np_array of cols + dynamic_param_col
    """
    window_lengths = [[5, 20, 67]]
    thresholds = [10]
    sensitivity = [0.5]
    return window_lengths, thresholds, sensitivity

df1= df.copy()
df2 = calc_tide_sig(df1,
                    cols_set=[['open','high','low']],
                     tide_window_threshold_func= dynamic_params_func,
                     dynamic_param_col = ["volume"])

In [ ]:
from backtesters import backtest
# ============================
# Backtest parameters
# ============================
signals = [f"tide"]



fee = 0.0001
# fee = 0
slippage = 0.0

min_holding_period = 0
max_holding_period = 1e6

long_equity = 5e5
long_notional=5e5
short_equity = 5e5
short_notional= 5e5

signal_function="default"
figsize = (20,15) #(20,29) # width, height
# figsize = (20,30)
# window =["2020-11-02","2022-08-23"] # - 0th backtest
# window =["2010-01-01","2023-03-23"] 
model_name= "Trend_Following"
# window =["2022-06-01","2023-12-31"] 
# model_name = "Mean_Reversion"


tradable_times = None #[["00:05", "23:50"]]
closing_session_times= None #[["23:50", "00:00"]]



timeframe = "1h"
sig_timeframe = f"1h"
kline_to_trade = f"close"
volume_to_trade= f"volume"



title = f"{timeframe} BTC spot | fees: {fee*1e4} bps"
file_name = f"{timeframe} BTC Tide"
# signal = "ES_USD_close_sig" 
# window = ["2021-01-01","2021-12-31"] 
df_sig = df.copy()
if signal_function == "default":
        df_sig["sig"] = df_sig[signals]
df_trade = df_sig.copy()

signals = df_trade.columns
df_backtested,df_trades,df_summary = backtest.backtest(model_name= model_name,
                                                df0=df_trade,
                                                timeframe=timeframe,
                                                kline_to_trade=kline_to_trade,
                                                volume_to_trade = volume_to_trade,
                                                tradable_times = tradable_times,
                                                closing_session_times = closing_session_times,
                                                position_sizing_to_trade=None,
                                                min_holding_period=min_holding_period/int(timeframe[:-1]),
                                                max_holding_period=max_holding_period/int(timeframe[:-1]),
                                                fee=fee,
                                                slippage=slippage,
                                                long_equity = long_equity,
                                                short_equity = short_equity,
                                                long_notional=long_notional,
                                                short_notional=short_notional,
                                                signals=signals, 
                                                signal_function=signal_function, 
                                                figsize=figsize, # width, height
                                                show_B=True,
                                                show_LS=True,
                                                title=title,
                                                file_name=file_name,
                                                plots=show_plots,
                                                diagnostics_verbose=False,
                                                trail_SL = None,
                                                trail_TP = None,
                                                trail_increment = None,
                                                N=365*25
                                                )

df = df_backtested.copy()

In [ ]:
df2

In [ ]:
df2[list(df2.filter(regex="tide").columns)+["close"]].tail(50).plot(secondary_y = "close")

### ii) Z signal V3

In [ ]:
import numpy as np
from signal_managers.indicators import calc_z_sig
# NOTE: window_threshold_func is very powerful, but can be very slow.
# its power lies in the fact that it can be used to calculate the threshold dynamically
# and this enables the threshold to be calculated based on the volatility of the instrument
# which allows for a more dynamic threshold calculation
# which is more robust to different market conditions
# and in terms of PNL performance, this means that the strategy is more robust to different market conditions
# and in terms of sharpe ratio this would go up with a more robust strategy

def dynamic_params_func(x, i):
    # Calculate the volatility of the input data
    volatility = np.std(x[:,0]) # This is too smooth to be vol triggers have to change for future use
    
    # Set the window lengths and threshold values based on the volatility
    if volatility < 0.01:
        windows = [288, 576]
        thresholds = [1.5, 2, 2.5]
    elif volatility < 0.05:
        windows = [144, 288]
        thresholds = [1, 1.5, 2]
    else:
        windows = [72, 144]
        thresholds = [0.5, 1, 1.5]
    
    return windows, thresholds

df1 = calc_z_sig(df, cols=['open','high','low','close'], window_threshold_func = dynamic_params_func)

In [ ]:
close_sigs = list(df1.filter(regex="close_sig").columns)
df1[close_sigs+["close"]].tail(100).plot(secondary_y = "close")

### iii) EWMAC V3

In [ ]:
import numpy as np
from signal_managers.indicators import calc_z_sig
# NOTE: window_threshold_func is very powerful, but can be very slow.
# its power lies in the fact that it can be used to calculate the threshold dynamically
# and this enables the threshold to be calculated based on the volatility of the instrument
# which allows for a more dynamic threshold calculation
# which is more robust to different market conditions
# and in terms of PNL performance, this means that the strategy is more robust to different market conditions
# and in terms of sharpe ratio this would go up with a more robust strategy

def dynamic_params_func(x, i):
    # Calculate the volatility of the input data
    volatility = np.std(x[:,0]) # This is too smooth to be vol triggers have to change for future use
    
    # Set the window lengths and threshold values based on the volatility
    if volatility < 0.01:
        windows = [288, 576]
        thresholds = [1.5, 2, 2.5]
    elif volatility < 0.05:
        windows = [144, 288]
        thresholds = [1, 1.5, 2]
    else:
        windows = [72, 144]
        thresholds = [0.5, 1, 1.5]
    
    return windows, thresholds

df1 = calc_z_sig(df, cols=['open','high','low','close'], window_threshold_func = dynamic_params_func)

In [ ]:
close_sigs = list(df1.filter(regex="close_sig").columns)
df1[close_sigs+["close"]].tail(100).plot(secondary_y = "close")

## b) Tide_Z dynamic parameters tests

In [ ]:
x=[[1,2,3,4,5,6,7,8,9],
    [6,7,8,9,1,2,3,4,5],
    [1,2,8,9,1,2,3,8,9]]
x = np.array(x)

i = 2
max_lookback = 2
y1 = x[i-max_lookback:2]
y2 = x[-max_lookback:,2]
print(f"y1 = x[i-max_lookback:2] : {y1}  \ny2 = x[-max_lookback:,2] : {y2}")

